## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

In [2]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [3]:
# do not change the code in the block below
# __________start of block__________

!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

--2024-11-21 19:07:29--  https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51581 (50K) [text/plain]
Saving to: ‘texts_holdout.json’

texts_holdout.json  100%[===================>]  50.37K  --.-KB/s    in 0.008s  

2024-11-21 19:07:29 (6.21 MB/s) - ‘texts_holdout.json’ saved [51581/51581]



In [4]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
# __________end of block__________

In [6]:
df

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


In [5]:
df[1].value_counts()

,count
1,
1,3610
0,3310


Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [ ]:
!pip install datasets

In [26]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import  Dataset
from sklearn.metrics import classification_report, f1_score, accuracy_score


In [22]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cuda'

In [11]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [24]:
num_labels = 2

In [47]:
# your beautiful experiments here
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels).to(device)
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

In [31]:
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import torch.optim as optim
from torch import nn

In [35]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [55]:
# Параметры
num_labels = 2  # Количество классов
max_len = 128
batch_size = 16
learning_rate = 2e-5
num_epochs = 2

# Создание датасетов
train_dataset = TextClassificationDataset(
    texts_train, y_train, tokenizer, max_len
)
val_dataset = TextClassificationDataset(
    texts_test, y_test, tokenizer, max_len
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Настройка оптимизатора только для последнего слоя
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4 )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',      # максимизируем accuracy
    patience=2,      # количество эпох без улучшения
    factor=0.5,      # во сколько раз уменьшаем learning rate
    verbose=True     # вывод информации об изменении lr
)

In [59]:
from tqdm.auto import tqdm

In [60]:
# Обучение
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Прогресс-бар для тренировочного цикла
    train_progress = tqdm(
        train_loader,
        desc=f'Epoch {epoch+1}/{num_epochs} (Train)',
        total=len(train_loader),
        unit='batch'
    )


    for batch in train_progress:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Обновление прогресс-бара
        train_progress.set_postfix({
            'loss': f'{loss.item():.4f}',
            'avg_loss': f'{total_loss / (train_progress.n + 1):.4f}'
        })

    # Закрытие прогресс-бара тренировки
    train_progress.close()

    model.eval()

    predictions = []
    true_labels = []

    # Прогресс-бар для валидационного цикла
    val_progress = tqdm(
        val_loader,
        desc=f'Epoch {epoch+1}/{num_epochs} (Validation)',
        total=len(val_loader),
        unit='batch'
    )

    with torch.no_grad():
        for batch in val_progress:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)

            preds = torch.argmax(outputs.logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Закрытие прогресс-бара валидации
    val_progress.close()

    # Метрики
    epoch_accuracy = accuracy_score(true_labels, predictions)
    epoch_val_loss = epoch_accuracy / len(val_loader)

    # Обновление scheduler на основе accuracy
    scheduler.step(epoch_accuracy)

    # Сохранение лучшей модели
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

    # Логирование результатов
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {total_loss/len(train_loader):.4f}')
    print(f'Val Loss: {epoch_val_loss:.4f}')
    print(f'Accuracy: {epoch_accuracy:.4f}')

Epoch 1/2 (Train):   0%|          | 0/313 [00:00<?, ?batch/s]

Epoch 1/2 (Validation):   0%|          | 0/120 [00:00<?, ?batch/s]

Epoch 1/2:
Train Loss: 0.4248
Val Loss: 0.0073
Accuracy: 0.8708


Epoch 2/2 (Train):   0%|          | 0/313 [00:00<?, ?batch/s]

Epoch 2/2 (Validation):   0%|          | 0/120 [00:00<?, ?batch/s]

Epoch 2/2:
Train Loss: 0.2056
Val Loss: 0.0072
Accuracy: 0.8693


In [61]:
# Валидация
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)

        preds = torch.argmax(outputs.logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Метрики
print("Accuracy:", accuracy_score(true_labels, predictions))
print("\nClassification Report:")
print(classification_report(true_labels, predictions))

Accuracy: 0.8692708333333333

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       917
           1       0.86      0.90      0.88      1003

    accuracy                           0.87      1920
   macro avg       0.87      0.87      0.87      1920
weighted avg       0.87      0.87      0.87      1920



#### Сдача взадания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к первому (положительному) классу

In [65]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
import json

with open('/content/texts_holdout.json', 'r') as f:
    holdout_texts = json.load(f)
len(holdout_texts)

500

In [66]:
y_holdout = np.zeros(len(holdout_texts), dtype=int)

holdout_dataset = TextClassificationDataset(
    holdout_texts, y_holdout, tokenizer, max_len
)

holdout_loader = DataLoader(holdout_dataset, batch_size=batch_size, shuffle=False)

In [75]:
# Функция для получения вероятностей
def get_probabilities(loader):
    model.eval()
    probabilities = []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            probs = torch.softmax(outputs.logits, dim=1)  # Получаем вероятности
            probabilities.extend(probs.cpu().numpy())  # Добавляем вероятности в список

    return probabilities

In [68]:
# Получение вероятностей для каждого набора
train_probs = get_probabilities(train_loader)
test_probs = get_probabilities(val_loader)
holdout_probs = get_probabilities(holdout_loader)

train_probs = [prob[1] for prob in train_probs]  # Извлекаем вероятность для класса с индексом 1
test_probs = [prob[1] for prob in test_probs]
holdout_probs = [prob[1] for prob in holdout_probs]

out_dict = {
    'train': [prob.tolist() for prob in train_probs],# list of length 5000 with probas
    'test':[prob.tolist() for prob in test_probs], # list of length 1920 with probas
    'holdout': [prob.tolist() for prob in holdout_probs]# list of length 500 with probas
}

# Вывод информации
print("Размеры вероятностей:")
print("Train:", len(out_dict['train']))
print("Test:", len(out_dict['test']))
print("Holdout:", len(out_dict['holdout']))

Размеры вероятностей:
Train: 5000
Test: 1920
Holdout: 500


Несколько `assert`'ов для проверки вашей посылки:

In [83]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["holdout"]) == 500
), "The predicted probas list length does not match the holdout set size"

Запустите код ниже для генерации посылки.

In [84]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

File saved to `submission_dict_hw_text_classification_with_bert.json`


На этом задание завершено. Поздравляем!